# Timing Model useage example

## Build a timing model 

In [1]:
from pint.models import get_model

One can build a timing model via get_model() method. get_model() will make the model according to the .par file. All the model components, delays or phases, will be stored in order. 

In [2]:
par = 'B1855+09_NANOGrav_dfg+12_TAI.par'
m = get_model(par)

To take a look what type of model component are in the model

In [3]:
m.component_types # We have delay component and phase component 

['DelayComponent', 'PhaseComponent']

TimingModel class stores the delay model components and phase components in the lists 

In [4]:
m.DelayComponent_list # Delays are stored in the DelayComponent_list

In [5]:
m.PhaseComponent_list # phases are stored in the PhaseComponent_list

To add a component:

In [6]:
from pint.models.astrometry import AstrometryEcliptic

In [7]:
a = AstrometryEcliptic() # init the AstrometryEcliptic instance

Add the new component instance into time model with order 3

In [8]:
m.add_component(a, 3)

In [9]:
m.DelayComponent_list # The new instance is added to delay component list 
                      # index 3

To remove a component is simple. Just use remove it from the list. You can map the component instance using name string via map_component() method

In [10]:
component, order, from_list, comp_type = \
    m.map_component('AstrometryEcliptic')
from_list.remove(component)

In [11]:
m.DelayComponent_list # AstrometryEcliptic is removed from delay list. 

To switch the order of a component, just change the index in the component list. 

In [12]:
# First map the component instance
component, order, from_list, comp_type = \
    m.map_component('JumpDelay')
# If one wants to move this component to a new order without swapping
from_list.remove(component)
from_list.insert(5, component)

In [13]:
m.DelayComponent_list

In [14]:
# If one wants to swap with other component
component, order, from_list, comp_type = \
    m.map_component('JumpDelay')
new_order = 2
from_list[order], from_list[new_order] = from_list[new_order], from_list[order]

In [15]:
m.DelayComponent_list

Delays will be computed in order. 

In [16]:
# First get the toas
from pint.toa import get_TOAs
t = get_TOAs('../tests/datafile/B1855+09_NANOGrav_dfg+12.tim')

INFO: Applying clock corrections. [pint.toa]
INFO: Observatory arecibo, loading clock file /Users/jingluo/Research_codes/PINT/pint/datafiles/time.dat [pint.observatory.topo_obs]
INFO: Observatory arecibo, loading GPS clock file /Users/jingluo/Research_codes/PINT/pint/datafiles/gps2utc.clk [pint.observatory.topo_obs]
INFO: Observatory arecibo, loading BIPM clock file /Users/jingluo/Research_codes/PINT/pint/datafiles/tai2tt_bipm2015.clk [pint.observatory.topo_obs]
INFO: Getting IERS params and computing TDBs. [pint.toa]
INFO: Computing TDB columns. [pint.toa]
INFO: Computing observatory positions and velocities. [pint.toa]
INFO: Compute positions and velocities of observatories and Earth (planets = False), using DE421 ephemeris [pint.toa]
INFO: Adding columns ssb_obs_pos ssb_obs_vel obs_sun_pos [pint.toa]


In [21]:
# compute the total delay
total_delay = m.delay(t.table)
total_delay

array([ 386.46306501,  292.74493628,  123.64677909,  -68.30001643,
       -391.49294556, -391.55211107, -317.02034107, -317.01903715,
       -317.01690251, -317.01021663, -172.22028083,  240.44391033,
        240.52758075,  366.04797969,  416.44705584,  416.38194674,
        238.67530959, -141.66866147, -368.26117845,  188.93718598,
        409.62695507,  360.61475497,  276.16176165,  -56.48086239,
       -403.64873011,  401.8358864 ,  373.61321114,    9.65651057,
       -325.51347386, -401.49974263,  386.45719088,  292.73906113,
        123.64090373,  -68.3058918 , -391.49882043, -391.55798594,
       -317.02621492, -317.024911  , -317.02277636, -317.01609048,
       -172.22615441,  240.43803662,  240.52170704,  366.04210564,
        416.44118117,  416.37607207,  238.66943427, -141.67453691,
       -368.26705358, -434.64202827, -318.54915314,  -59.74843877,
        188.93131205,  409.62108048,  360.60887966,  143.43395126,
        415.55294851,  276.15588603,  -56.48673829, -403.65460

One can get the delay upto some component. For example, I want to the delay computation stop at jump delay.

In [19]:
to_jump_delay = m.delay(t.table, cutoff_component='JumpDelay')
to_jump_